In [1]:
import importlib
import modelsandbox
importlib.reload(modelsandbox)
from modelsandbox import Model

In [3]:
model = Model()
model.root.add_layer()
model.root.add_layer().add_sequence().add_layer()
layer = model.root.add_layer().add_sequence().add_layer()
layer.add_sequence()
layer.add_sequence()
model.root.add_layer()

In [4]:
model.processors

[[], [[[]]], [[[[], []]]], []]

In [6]:
model[2][0][1].processors

[]

# Cost example

In [5]:
import importlib
import modelsandbox
importlib.reload(modelsandbox)
from modelsandbox import Model

# Initialize the model class
model = Model()

# Add a layer to the model to compute airline ticket cost
model[0].set_label('Compute ticket cost')

# Add the process schema to the model
model[0].add_schema({
    "label": "ticket_cost",
    "params": ["destination", "airline_class"],
    "actions": ["get", "get"],
    "data": {
        "Chicago": {
            "Economy": 220,
            "Business": 450,
            "First": 785
        },
        "Los Angeles": {
            "Economy": 365,
            "Business": 520,
            "First": 965
        }
    }
})

# Add a layer to the model to compute additional costs
model[1].set_label('Compute expenses')

# Add a processor to compute travel cost
@model[1].add_wrapped(tags=['__cost'])
def travel_cost(number_of_travelers, ticket_cost):
    """
    Compute total travel cost for all travelers.
    """
    return number_of_travelers * ticket_cost

# Add processor to compute lodging cost
@model[1].add_wrapped(tags=['__cost'])
def lodging_cost(number_of_travelers, nightly_cost, number_of_nights):
    """
    Compute total lodging cost for all travelers.
    """
    return number_of_travelers * nightly_cost * number_of_nights

# Add processor to compute per diem
@model[1].add_wrapped(tags=['__cost'])
def per_diem_cost(number_of_travelers, number_of_nights, per_diem):
    """
    Compute total per diem cost for all travelers.
    """
    return number_of_travelers * number_of_nights * per_diem

# Add a layer to the model to aggregate costs
model[2].set_label('Aggregate expenses')

# Add processor to compute total trip cost
@model[2].add_wrapped()
def total_trip_cost(travel_cost, lodging_cost, per_diem_cost, __cost):
    """
    Compute total trip cost for all travelers.
    """
    return travel_cost + lodging_cost + per_diem_cost

model.structure

{'Compute ticket cost': {'ticket_cost': <modelsandbox.model.processors.SchemaProcessor at 0x7f30fc0ba830>},
 'Compute expenses': {'travel_cost': <modelsandbox.model.processors.FunctionProcessor at 0x7f30fc0fe560>,
  'lodging_cost': <modelsandbox.model.processors.FunctionProcessor at 0x7f30fc0fe8c0>,
  'per_diem_cost': <modelsandbox.model.processors.FunctionProcessor at 0x7f30fc0ea230>},
 'Aggregate expenses': {'total_trip_cost': <modelsandbox.model.processors.FunctionProcessor at 0x7f30fc0ea800>}}

In [12]:
model.tagged

{'__cost': [<modelsandbox.model.processors.FunctionProcessor at 0x7f30fc0fe560>,
  <modelsandbox.model.processors.FunctionProcessor at 0x7f30fc0ea230>]}

In [6]:
model.analyze(
    airline_class="Business",
    destination="Chicago",
    nightly_cost=185,
    number_of_nights=4,
    number_of_travelers=3,
    per_diem=72,
)

{'airline_class': 'Business',
 'destination': 'Chicago',
 'nightly_cost': 185,
 'number_of_nights': 4,
 'number_of_travelers': 3,
 'per_diem': 72,
 'ticket_cost': 450,
 'travel_cost': 1350,
 'lodging_cost': 2220,
 'per_diem_cost': 864,
 '__cost': {'travel_cost': 1350, 'lodging_cost': 2220, 'per_diem_cost': 864},
 'total_trip_cost': 4434}

# Scratch

In [1]:
from example import model

In [4]:
model.analyze(
    airline_class="Business",
    destination="Chicago",
    nightly_cost=185,
    number_of_nights=4,
    number_of_travelers=3,
    per_diem=72,
    ticket_cost=500
)

{'airline_class': 'Business',
 'destination': 'Chicago',
 'nightly_cost': 185,
 'number_of_nights': 4,
 'number_of_travelers': 3,
 'per_diem': 72,
 'ticket_cost': 450,
 'travel_cost': 1350,
 'lodging_cost': 2220,
 'per_diem_cost': 864,
 'total_trip_cost': 4434}

In [6]:
model.parameters

['airline_class',
 'destination',
 'nightly_cost',
 'number_of_nights',
 'number_of_travelers',
 'per_diem']

In [1]:
from sample_model import model

In [2]:
model.analyze(
    aadt = 580,
    curve_length = 0.2,
    curve_radius = 800,
    lane_width = 11,
    length = 0.6,
    observed_kabco = 3.4,
    shoulder_type = 'gravel',
    shoulder_width = 4,
    spiral = 'both'
)

{'aadt': 580,
 'curve_length': 0.2,
 'curve_radius': 800,
 'lane_width': 11,
 'length': 0.6,
 'observed_kabco': 3.4,
 'shoulder_type': 'gravel',
 'shoulder_width': 4,
 'spiral': 'both',
 'n_kabco': 0.9297629371556028,
 'overdispersion': 0.3933333333333333,
 'af_lane_width': 1.0145,
 'af_shoulder_width': 1.0346250000000001,
 'af_shoulder_type': 1.01,
 'af_horizontal_curve': 1.2846774193548387,
 'af_total': 1.3619165077968751,
 'predicted_kabco': 1.266259492449924,
 'expected_kabco': 1.9756661526181518}